In [1]:
#文本清洗
import pandas as pd  
import re  

# 读取数据  
df = pd.read_csv("fault_data_labeled.csv")  
# 去除特殊符号、数字，只保留中文/英文  
df["fault_desc_clean"] = df["fault_desc"].apply(lambda x: re.sub(r"[^\u4e00-\u9fa5a-zA-Z]", " ", str(x)))  
# 去除停用词（如“的”“我”“怎么办”等无意义词）  
stopwords = ["的", "我", "怎么办", "电脑", "软件", "故障"]  # 可自定义或用现成停用词表  
df["fault_desc_clean"] = df["fault_desc_clean"].apply(lambda x: " ".join([w for w in x.split() if w not in stopwords]))  

In [3]:
print(df["fault_desc_clean"])

0                  Windows 开机后卡在登录界面 输入密码没反应 重启也没用
1                 MacBook Pro更新系统后 触控板失灵 点击和滑动都没反应
2                          电脑突然蓝屏 代码 x B 重启后还是循环蓝屏
3              Windows 提示 无法验证此驱动程序的发布者 导致显卡驱动安装失败
4             Mac系统升级到Ventura后 外接显示器显示黑屏 只识别到分辨率 x
                          ...                     
133                Excel数据透视表刷新时提示 引用无效 数据源未被移动或删除
134                 钉钉直播时共享屏幕 观众看到的画面是静止的 自己这边显示正常
135            Photoshop保存为JPG格式时 提示 无法完成请求 因为程序错误
136             Visio插入的CAD图纸显示不全 只能看到部分内容 CAD文件完整
137    Python使用pandas处理大数据时 提示 MemoryError 电脑内存 GB
Name: fault_desc_clean, Length: 138, dtype: object


In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer  

tfidf = TfidfVectorizer(max_features=200)  # 保留200个最关键的词  
X = tfidf.fit_transform(df["fault_desc_clean"]).toarray()  # 特征矩阵  
y = df["fault_type"]  # 标签  

In [11]:
print(X)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


### 上面特征值打印出来全是0，有以下可能
1.数据预处理不充分（最可能）
如果 fault_desc_clean 列中的文本没有正确分词（尤其是中文），TfidfVectorizer 会将每个字符视为一个 “词”，导致特征稀疏度过高。
示例：
原文本："Windows系统蓝屏"
未分词时被拆分为：["W", "i", "n", "d", "o", "w", "s", "系", "统", "蓝", "屏"]
每个字符单独出现频率极低，TF-IDF 值接近 0。
2.特征维度过高，样本稀疏
max_features=1000 可能保留了太多不相关的特征，而你的文本数据较少（150 条），导致大部分样本在这 1000 个特征上的值为 0。
3.文本清洗过度
如果在 fault_desc_clean 中删除了所有有意义的词，只剩下空字符串或停用词，TF-IDF 会生成全零向量。

In [3]:
#对中文文本进行分词
from sklearn.feature_extraction.text import TfidfVectorizer 
import jieba

# 分词并添加空格（TfidfVectorizer默认按空格分词）
df["fault_desc_tokenized"] = df["fault_desc_clean"].apply(
    lambda x: " ".join(jieba.cut(x))
)

# 重新创建并训练TF-IDF向量器
tfidf = TfidfVectorizer(max_features=200)
X = tfidf.fit_transform(df["fault_desc_tokenized"]).toarray()
y = df["fault_type"]  # 标签  

# 保存TF-IDF向量器
import joblib
joblib.dump(tfidf, "tfidf_vectorizer.pkl")

# 检查非零元素比例（验证是否解决问题）
non_zero_ratio = (X != 0).sum() / X.size
print(f"非零元素比例: {non_zero_ratio:.4f}")

非零元素比例: 0.0351


In [9]:
print("前20个特征词:", tfidf.get_feature_names_out()[:20])

前20个特征词: ['adobe' 'air' 'autocad' 'cad' 'chrome' 'cmos' 'cpu' 'defender' 'dns'
 'excel' 'gb' 'ip' 'ip地址' 'jpg' 'kb' 'mac' 'macbook' 'oa' 'office'
 'outlook']


In [12]:

from sklearn.model_selection import train_test_split  
#测试集占总数据比例20%
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)  
#支持向量机
from sklearn.svm import SVC  
model_svm = SVC(kernel="linear",C=10)  # 线性核适合文本  
model_svm.fit(X_train, y_train)  
# 保存模型
joblib.dump(model_svm, "fault_classifier.pkl")

['fault_classifier.pkl']

In [13]:
from sklearn.metrics import accuracy_score, confusion_matrix  
# 以SVM为例  
y_pred = model_svm.predict(X_test)  
print(f"准确率：{accuracy_score(y_test, y_pred):.2f}")  
print("混淆矩阵：\n", confusion_matrix(y_test, y_pred))  


准确率：0.71
混淆矩阵：
 [[7 1 0 2]
 [1 4 0 0]
 [0 2 4 0]
 [2 0 0 5]]


In [ ]:
from flask import Flask, request, jsonify  
app = Flask(__name__)  
model = joblib.load("fault_classifier.pkl")  
tfidf = joblib.load("tfidf_vectorizer.pkl")  
solutions = pd.read_csv("solutions.csv")  
@app.route("/predict", methods=["POST"])  
def predict():  
    user_input = request.json.get("fault_desc")  # 用户输入的故障描述
    # 预处理  
    input_clean = re.sub(r"[^\u4e00-\u9fa5a-zA-Z]", " ", user_input)  
    input_tfidf = tfidf.transform([input_clean]).toarray()  
    # 预测故障类型  
    pred_type = model.predict(input_tfidf)[0]  
    # 匹配解决方案  
    solution = solutions[solutions["fault_type"] == pred_type]["solution"].values[0]  
    return jsonify({  
        "fault_type": pred_type,  
        "solution": solution  
    })  


In [15]:
 
app.run(debug=True)  # 本地运行，访问http://127.0.0.1:5000  

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1

C:\Users\Wendy Xue\AppData\Roaming\Python\Python312\site-packages\IPython\core\interactiveshell.py:3675: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
